In [18]:
import keras
from keras.layers import LSTM, Dropout, Dense
import tensorflow as tf
import numpy as np
import pandas as pd

https://github.com/abulbasar/neural-networks/blob/master/Keras%20-%20Multivariate%20time%20series%20classification%20using%20LSTM.ipynb

https://github.com/akshaykanehitlu/Credit-Card-Fraud-Detection-ML/blob/main/creditcard.csv

https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/

In [19]:
data = pd.read_csv("creditcard.csv.gz")

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [21]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [22]:
rec_count = len(data)
rec_count

284807

In [43]:
sequence_length = 10

In [44]:
def generate_data(X, y, sequence_length = sequence_length, step = 1):
    X_local = []
    y_local = []
    for start in range(0, len(data) - sequence_length, step):
        end = start + sequence_length
        X_local.append(X[start:end])
        y_local.append(y[end-1])
    return np.array(X_local), np.array(y_local)

X_sequence, y = generate_data(data.loc[:, "V1":"V28"].values, data.Class)

In [45]:
X_sequence.shape, y.shape

((284797, 10, 28), (284797,))

In [26]:
model = keras.Sequential()
model.add(LSTM(100, input_shape = (10, 28)))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy"
              , metrics=[keras.metrics.binary_accuracy]
              , optimizer="adam")

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               51600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 51,701
Trainable params: 51,701
Non-trainable params: 0
_________________________________________________________________


In [27]:
training_size = int(len(X_sequence) * 0.7)

In [28]:
X_train, y_train = X_sequence[:training_size], y[:training_size]
X_test, y_test = X_sequence[training_size:], y[training_size:]

In [29]:
X_train.shape

(199357, 10, 28)

In [30]:
model.fit(X_train, y_train, batch_size=64, epochs=10) # epoch -> 50

Epoch 1/10
3115/3115 [==============================] - 78s 23ms/step - loss: 0.0080 - binary_accuracy: 0.9977
Epoch 2/10
3115/3115 [==============================] - 87s 28ms/step - loss: 0.0035 - binary_accuracy: 0.9993
Epoch 3/10
3115/3115 [==============================] - 78s 25ms/step - loss: 0.0035 - binary_accuracy: 0.9993
Epoch 4/10
3115/3115 [==============================] - 87s 28ms/step - loss: 0.0032 - binary_accuracy: 0.9993
Epoch 5/10
3115/3115 [==============================] - 100s 32ms/step - loss: 0.0032 - binary_accuracy: 0.99930s - loss: 0.0032 - binary_accuracy: 0.
Epoch 6/10
3115/3115 [==============================] - 95s 31ms/step - loss: 0.0029 - binary_accuracy: 0.9993 0s - loss: 0.0029 - binary_accu
Epoch 7/10
3115/3115 [==============================] - 87s 28ms/step - loss: 0.0026 - binary_accuracy: 0.9994
Epoch 8/10
3115/3115 [==============================] - 75s 24ms/step - loss: 0.0024 - binary_accuracy: 0.9994
Epoch 9/10
3115/3115 [==================

In [31]:
model.evaluate(X_test, y_test)

2670/2670 [==============================] - 18s 6ms/step - loss: 0.0036 - binary_accuracy: 0.9996


[0.003633186686784029, 0.9995903372764587]

In [32]:
y_test_prob = model.predict(X_test, verbose=1)

2670/2670 [==============================] - 17s 6ms/step


In [40]:
y_test_pred = np.where(y_test_prob > 0.7, 1, 0)

In [41]:
from sklearn.metrics import confusion_matrix

In [42]:
confusion_matrix(y_test, y_test_pred)

array([[85331,     1],
       [   38,    70]], dtype=int64)